# Programming Homework 2 
@author: Yijin Xiang  
@Created on Sunday June 7

In [105]:
# Load in the fasta file
# Set the working directory
import os
os.chdir('/Users/xiangyijin/Desktop/genetics/')
os.getcwd()

#Create function to read in the fasta format data (put line by line data into a string)
def readGenome(filename):
    output = ''
    with open(filename,'r') as f: #'r'represents read only while 'w' represents write 
        for line in f:
            if line[0] != '>':
                output += line.rstrip()
    return output
genome = readGenome('chr1.GRCh38.excerpt.fasta')
genome[:200] # check the input

'TTGAATGCTGAAATCAGCAGGTAATATATGATAATAGAGAAAGCTATCCCGAAGGTGCATAGGTCAACAATACTTGAGCCTAACTCAGTAGATCCTAAAAGAAAGCAATTTTTGCTGCTAACCTAACATTTCACAATGTCTGGAGACATTTACAGTTCCCACAACCTATGGCAGTTACTGGCATCTACTAGAGGTCAGAG'

## Question 1-3
To match the string **GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG** (derived from human Alu sequences) to the excerpt of human chromosome 1?

In [104]:
p ="GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG"

### **Question 1** How many alignments does the naive exact matching algorithm try?

In [106]:

len(genome) - len(p) + 1

799954

### **Question 2** How many character comparisons does the naive exact matching algorithm try?

In [112]:
num_comparison = 0
for i in range(len(genome)-len(p)+1):
    for j in range(len(p)):
        num_comparison += 1
        if p[j] != genome[i+j] :
            break
print(num_comparison)    

984143


### Question 3 How many alignments does Boyer-Moore try?

In [108]:
import string

def z_array(s):
    """ Use Z algorithm (Gusfield theorem 1.4.1) to preprocess s """
    assert len(s) > 1
    z = [len(s)] + [0] * (len(s)-1)
    # Initial comparison of s[1:] with prefix
    for i in range(1, len(s)):
        if s[i] == s[i-1]:
            z[1] += 1
        else:
            break
    r, l = 0, 0
    if z[1] > 0:
        r, l = z[1], 1
    for k in range(2, len(s)):
        assert z[k] == 0
        if k > r:
            # Case 1
            for i in range(k, len(s)):
                if s[i] == s[i-k]:
                    z[k] += 1
                else:
                    break
            r, l = k + z[k] - 1, k
        else:
            # Case 2
            # Calculate length of beta
            nbeta = r - k + 1
            zkp = z[k - l]
            if nbeta > zkp:
                # Case 2a: Zkp wins
                z[k] = zkp
            else:
                # Case 2b: Compare characters just past r
                nmatch = 0
                for i in range(r+1, len(s)):
                    if s[i] == s[i - k]:
                        nmatch += 1
                    else:
                        break
                l, r = k, r + nmatch
                z[k] = r - k + 1
    return z


def n_array(s):
    """ Compile the N array (Gusfield theorem 2.2.2) from the Z array """
    return z_array(s[::-1])[::-1]


def big_l_prime_array(p, n):
    """ Compile L' array (Gusfield theorem 2.2.2) using p and N array.
        L'[i] = largest index j less than n such that N[j] = |P[i:]| """
    lp = [0] * len(p)
    for j in range(len(p)-1):
        i = len(p) - n[j]
        if i < len(p):
            lp[i] = j + 1
    return lp


def big_l_array(p, lp):
    """ Compile L array (Gusfield theorem 2.2.2) using p and L' array.
        L[i] = largest index j less than n such that N[j] >= |P[i:]| """
    l = [0] * len(p)
    l[1] = lp[1]
    for i in range(2, len(p)):
        l[i] = max(l[i-1], lp[i])
    return l


def small_l_prime_array(n):
    """ Compile lp' array (Gusfield theorem 2.2.4) using N array. """
    small_lp = [0] * len(n)
    for i in range(len(n)):
        if n[i] == i+1:  # prefix matching a suffix
            small_lp[len(n)-i-1] = i+1
    for i in range(len(n)-2, -1, -1):  # "smear" them out to the left
        if small_lp[i] == 0:
            small_lp[i] = small_lp[i+1]
    return small_lp


def good_suffix_table(p):
    """ Return tables needed to apply good suffix rule. """
    n = n_array(p)
    lp = big_l_prime_array(p, n)
    return lp, big_l_array(p, lp), small_l_prime_array(n)


def good_suffix_mismatch(i, big_l_prime, small_l_prime):
    """ Given a mismatch at offset i, and given L/L' and l' arrays,
        return amount to shift as determined by good suffix rule. """
    length = len(big_l_prime)
    assert i < length
    if i == length - 1:
        return 0
    i += 1  # i points to leftmost matching position of P
    if big_l_prime[i] > 0:
        return length - big_l_prime[i]
    return length - small_l_prime[i]


def good_suffix_match(small_l_prime):
    """ Given a full match of P to T, return amount to shift as
        determined by good suffix rule. """
    return len(small_l_prime) - small_l_prime[1]


def dense_bad_char_tab(p, amap):
    """ Given pattern string and list with ordered alphabet characters, create
        and return a dense bad character table.  Table is indexed by offset
        then by character. """
    tab = []
    nxt = [0] * len(amap)
    for i in range(0, len(p)):
        c = p[i]
        assert c in amap
        tab.append(nxt[:])
        nxt[amap[c]] = i+1
    return tab


class BoyerMoore(object):
    """ Encapsulates pattern and associated Boyer-Moore preprocessing. """
    
    def __init__(self, p, alphabet='ACGT'):
        self.p = p
        self.alphabet = alphabet
        # Create map from alphabet characters to integers
        self.amap = {}
        for i in range(len(self.alphabet)):
            self.amap[self.alphabet[i]] = i
        # Make bad character rule table
        self.bad_char = dense_bad_char_tab(p, self.amap)
        # Create good suffix rule table
        _, self.big_l, self.small_l_prime = good_suffix_table(p)
    
    def bad_character_rule(self, i, c):
        """ Return # skips given by bad character rule at offset i """
        assert c in self.amap
        ci = self.amap[c]
        assert i > (self.bad_char[i][ci]-1)
        return i - (self.bad_char[i][ci]-1)
    
    def good_suffix_rule(self, i):
        """ Given a mismatch at offset i, return amount to shift
            as determined by (weak) good suffix rule. """
        length = len(self.big_l)
        assert i < length
        if i == length - 1:
            return 0
        i += 1  # i points to leftmost matching position of P
        if self.big_l[i] > 0:
            return length - self.big_l[i]
        return length - self.small_l_prime[i]
    
    def match_skip(self):
        """ Return amount to shift in case where P matches T """
        return len(self.small_l_prime) - self.small_l_prime[1]

In [109]:
p_bm = BoyerMoore(p, alphabet='ACGT')
def boyer_moore(p,p_bm,t):
    i = 0
    occurences = []
    alignments = 0 # number of allignments skipped
    while i < len(t) - len(p) + 1:
        shift = 1
        mismatched = False
        for j in range (len(p)-1, -1, -1):
            if p[j] != t[i+j]:
                skip_bc = p_bm.bad_character_rule(j,t[i+j]) # missmacth happens at offset j and the correct character should be t[i+j]
                skip_gs = p_bm.good_suffix_rule(j)
                shift = max(shift, skip_bc, skip_gs)
                mismatched = True
                break
        if not mismatched:
            occurences.append(i)
            skip_gs = p_bm.match_skip() #shift all p to start a new comparasion
            shift = max(shift, skip_gs)
        i += shift 
        alignments += 1
    return occurences,alignments

In [110]:
boyer_moore(p, p_bm, genome)[1]

127974

## Question 4-5
Write a function that, given a length-24 pattern P and given an Index object built on 8-mers, finds all approximate occurrences of P within T with up to 2 mismatches

### Question 4
How many times does the string GGCGCGGTGGCTCACGCCTGTAAT, which is derived from a human Alu sequence, occur with up to 2 substitutions in the excerpt of human chromosome 1? (Don't consider reverse complements here.)

In [83]:
# Appoximate matching (with boyermoor preprocessing of the pattern)
def approximate_match(p,t,n): #n- maximum number of mismatch
    segement_length = 8
    all_matches = set()
    hits = 0
    for i in range(n+1):
        start = i * segement_length
        end = min((i+1)*segement_length, len(p)) # make sure we don't pass the length of p 
        p_bm = BoyerMoore(p[start:end], alphabet = 'ACGT') # preprocessing p
        matches = boyer_moore(p[start:end], p_bm, t)[0] # return a list of offset for occurance
        hits += len(matches) # Gain the number of hits for each segement
        # Verification
        for m in matches:
            if m < start or m - start + len(p) > len(t):
                # avoid p runs off the beginning(eg.P=CGTCC,T=GTCCA,m=0, start =1)
                #or pasts the end of t
                continue
            mismatches = 0
            # Verify the sequence before the match
            for j in range(start):
                if not p[j] == t[m-start+j]:
                    mismatches += 1
                    if mismatches > n:
                        break
            # Verify the sequence after the match
            for j in range(end, len(p)):
                if not p[j] == t[m-start+j]:
                    mismatches += 1
                    if mismatches > n:
                        break
            if mismatches <= n:
                all_matches.add(m - start) # add the offset on the strend
    return list(all_matches),hits
        

In [84]:
p= "GGCGCGGTGGCTCACGCCTGTAAT"
t = genome
match = approximate_match(p, t, 2)[0]
match.sort()
print(match)
len(match)

[56922, 84641, 147558, 160162, 160729, 191452, 262042, 273669, 364263, 421221, 429299, 465647, 551134, 635931, 657496, 681737, 717706, 724927, 747359]


19

In [86]:
hits = approximate_match(p, t, 2)[1]
hits

90

## Queston 6
Write a function that, given a length-24 pattern P and given a SubseqIndex object built with k = 8 and ival = 3, finds all approximate occurrences of P within T with up to 2 mismatches.

In [77]:
import bisect
   
class SubseqIndex(object):
    """ Holds a subsequence index for a text T """
    
    def __init__(self, t, k, ival):
        """ Create index from all subsequences consisting of k characters
            spaced ival positions apart.  E.g., SubseqIndex("ATAT", 2, 2)
            extracts ("AA", 0) and ("TT", 1). """
        self.k = k  # num characters per subsequence extracted
        self.ival = ival  # space between them; 1=adjacent, 2=every other, etc
        self.index = []
        self.span = 1 + ival * (k - 1) # The last 
        for i in range(len(t) - self.span + 1):  # for each subseq
            self.index.append((t[i:i+self.span:ival], i))  # add (subseq, offset)
        self.index.sort()  # alphabetize by subseq
    
    def query(self, p):
        """ Return index hits for first subseq of p """
        hits = []
        for j in range(3):
            subseq = p[j:j+self.span:self.ival]  # query with first subseq
            i = bisect.bisect_left(self.index, (subseq, -1))  # binary search
            while i < len(self.index):  # collect matching index entries
                if self.index[i][0] != subseq:
                    break
                hits.append(self.index[i][1]-j) # let the hits record the offset of the initial of the pattern
                i += 1
        return hits
    

In [134]:
def query_subseq(p, t, subseq_ind):
    offsets = set()
    for i in subseq_ind.query(p):
        mismatches = 0
        for j in range(len(p)):
            if p[j] != t[i+j]:
                mismatches += 1 
                if mismatches > 2:
                    break
        if mismatches <= 2:
                offsets.add(i)
    return list(offsets)

In [135]:
p= "GGCGCGGTGGCTCACGCCTGTAAT"
t = genome
subseq_ind = SubseqIndex(t, 8, 3)
hit=subseq_ind.query(p) 
len(hit)# number of hits
query_subseq(p,t,subseq_ind) # offset of occurance

[84641,
 273669,
 147558,
 364263,
 681737,
 717706,
 465647,
 747359,
 657496,
 56922,
 635931,
 191452,
 262042]